#### 探究磁盘数据与发生故障的相关性

##### 导入包

In [1]:
import os
import zipfile
import pandas as pd
import gc
import glob
from sqlalchemy import create_engine
from datetime import timedelta

##### 读取数据集

In [2]:
#读取错误标签数据集
fault_tag_df = pd.read_csv('fault_tag_data.csv')
fault_tag_df = fault_tag_df.sort_values('fault_time')
fault_tag_df['fault_time'] = pd.to_datetime(fault_tag_df['fault_time'])
# 将同一个模型的数据归结到一起
fault_tag_A_df = fault_tag_df[fault_tag_df['model'] == 1]
fault_tag_B_df = fault_tag_df[fault_tag_df['model'] == 2]
fault_tag_A_df = fault_tag_A_df[['serial_number','fault_time']]
fault_tag_B_df = fault_tag_B_df[['serial_number','fault_time']]
fault_tag_A_df

,serial_number,fault_time
2320,disk_125207,2017-07-02
1214,disk_133302,2017-07-04
2004,disk_72870,2017-07-05
2384,disk_19440,2017-07-05
2326,disk_127633,2017-07-06
...,...,...
37,disk_114679,2018-12-31
509,disk_113846,2018-12-31
2298,disk_114247,2018-12-31
566,disk_143012,2018-12-31


In [3]:
# 读取csv文件
smartlog_data_df = pd.read_csv('smartlog_data_201812.csv')
# 删除空值
smartlog_data_df = smartlog_data_df.dropna(axis=1, how='all')

In [4]:
smartlog_data_A_df = smartlog_data_df[smartlog_data_df['model'] == 1]
smartlog_data_B_df = smartlog_data_df[smartlog_data_df['model'] == 2]
thresh_A = int(smartlog_data_A_df.shape[0] * 0.8)  # 这里设置阈值为总行数的80%
thresh_B = int(smartlog_data_B_df.shape[0] * 0.8)  # 这里设置阈值为总行数的80%
# 删除空值超过80%的列
smartlog_data_A_df = smartlog_data_A_df.dropna(thresh = thresh_A, axis=1)
smartlog_data_B_df = smartlog_data_B_df.dropna(thresh = thresh_B, axis=1)
smartlog_data_B_df

,Unnamed: 0,serial_number,manufacturer,model,smart_1_normalized,smart_1raw,smart_3_normalized,smart_3raw,smart_4_normalized,smart_4raw,...,smart_198raw,smart_199_normalized,smart_199raw,smart_240_normalized,smart_240raw,smart_241_normalized,smart_241raw,smart_242_normalized,smart_242raw,dt
2590859,247449,disk_54043,A,2,77.0,52066760.0,86.0,0.0,100.0,48.0,...,0.0,200.0,0.0,100.0,2.373871e+14,100.0,1.338755e+12,100.0,1.753766e+12,20181203
2590860,247450,disk_54044,A,2,72.0,16337240.0,87.0,0.0,100.0,18.0,...,0.0,200.0,0.0,100.0,1.391140e+13,100.0,1.323625e+12,100.0,1.822320e+12,20181210
2590861,247451,disk_54046,A,2,75.0,33993312.0,86.0,0.0,100.0,42.0,...,0.0,200.0,0.0,100.0,6.953552e+13,100.0,1.344010e+12,100.0,1.853416e+12,20181221
2590862,247452,disk_54046,A,2,82.0,142391728.0,86.0,0.0,100.0,42.0,...,0.0,200.0,0.0,100.0,1.869642e+14,100.0,1.314417e+12,100.0,1.803971e+12,20181209
2590863,247453,disk_54047,A,2,79.0,84753208.0,86.0,0.0,100.0,28.0,...,0.0,200.0,0.0,100.0,1.465486e+14,100.0,2.002543e+11,100.0,4.974017e+10,20181212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5858523,585848,disk_63107,A,2,100.0,13758.0,86.0,0.0,100.0,22.0,...,0.0,200.0,0.0,100.0,3.088081e+12,100.0,1.649433e+10,100.0,1.655887e+08,20181229
5858524,585849,disk_63109,A,2,81.0,135584328.0,87.0,0.0,100.0,12.0,...,0.0,200.0,0.0,100.0,1.542538e+14,100.0,3.237124e+11,100.0,1.863804e+11,20181202
5858525,585850,disk_6311,A,2,81.0,116649656.0,88.0,0.0,100.0,16.0,...,0.0,200.0,0.0,100.0,2.797785e+14,100.0,1.220101e+11,100.0,2.990756e+10,20181202
5858526,585851,disk_63111,A,2,84.0,233072600.0,86.0,0.0,100.0,15.0,...,0.0,200.0,1.0,100.0,3.711711e+13,100.0,5.204338e+10,100.0,1.197793e+12,20181227


In [5]:
# 筛选所有在该月发生故障的磁盘信息
smartlog_data_201812_fault_A_df = smartlog_data_A_df[smartlog_data_A_df['serial_number'].isin(fault_tag_A_df['serial_number'])].copy()
smartlog_data_201812_fault_B_df = smartlog_data_B_df[smartlog_data_B_df['serial_number'].isin(fault_tag_B_df['serial_number'])].copy()
# 将dt的格式转化成时间序列，方便和故障时间进行比较
smartlog_data_201812_fault_A_df.loc[:, 'dt'] = pd.to_datetime(smartlog_data_201812_fault_A_df['dt'], format='%Y%m%d')
smartlog_data_201812_fault_B_df.loc[:, 'dt'] = pd.to_datetime(smartlog_data_201812_fault_B_df['dt'], format='%Y%m%d')
# 混合数据集，方便比较
smartlog_data_201812_fault_A_df = pd.merge(smartlog_data_201812_fault_A_df, fault_tag_A_df, on='serial_number')
smartlog_data_201812_fault_B_df = pd.merge(smartlog_data_201812_fault_B_df, fault_tag_B_df, on='serial_number')
# 计算每个磁盘的最早一次观察时间
smartlog_data_201812_fault_A_df['first_observation_time'] = smartlog_data_201812_fault_A_df.groupby('serial_number')['dt'].transform('min')
smartlog_data_201812_fault_B_df['first_observation_time'] = smartlog_data_201812_fault_B_df.groupby('serial_number')['dt'].transform('min')
# 计算故障时间与最早一次观察时间的差值
smartlog_data_201812_fault_A_df['time_diff'] = (smartlog_data_201812_fault_A_df['fault_time'] - smartlog_data_201812_fault_A_df['first_observation_time']).dt.days
smartlog_data_201812_fault_B_df['time_diff'] = (smartlog_data_201812_fault_B_df['fault_time'] - smartlog_data_201812_fault_B_df['first_observation_time']).dt.days
# 筛选出那些最早的故障时间距离最后一次观察时间大于5天的磁盘的数据
smartlog_data_201812_fault_A_df = smartlog_data_201812_fault_A_df[smartlog_data_201812_fault_A_df['time_diff'] >= 6]
smartlog_data_201812_fault_B_df = smartlog_data_201812_fault_B_df[smartlog_data_201812_fault_B_df['time_diff'] >= 6]
# 筛选故障磁盘信息，信息只取故障发生之前的，防止磁盘维修好后，继续投入使用，重新投入使用的磁盘的数据是健康数据。
smartlog_data_201812_fault_A_df = smartlog_data_201812_fault_A_df[(pd.Timedelta(days=0) <= (smartlog_data_201812_fault_A_df['fault_time']-smartlog_data_201812_fault_A_df['dt'])) & ((smartlog_data_201812_fault_A_df['fault_time'] - smartlog_data_201812_fault_A_df['dt']) <= pd.Timedelta(days=6))]
smartlog_data_201812_fault_B_df = smartlog_data_201812_fault_B_df[(pd.Timedelta(days=0) <= (smartlog_data_201812_fault_B_df['fault_time']-smartlog_data_201812_fault_B_df['dt'])) & ((smartlog_data_201812_fault_B_df['fault_time'] - smartlog_data_201812_fault_B_df['dt']) <= pd.Timedelta(days=6))]
# 删除与故障无关的列
columns_to_drop = ["fault_time", "first_observation_time", "time_diff"]
smartlog_data_201812_fault_A_df = smartlog_data_201812_fault_A_df.drop(columns_to_drop, axis=1)
smartlog_data_201812_fault_B_df = smartlog_data_201812_fault_B_df.drop(columns_to_drop, axis=1)
# 填充:使用下一个观测值来填充当前的缺失值
smartlog_data_201812_fault_A_df = smartlog_data_201812_fault_A_df.bfill()
smartlog_data_201812_fault_B_df = smartlog_data_201812_fault_B_df.bfill()
# 添加标签
smartlog_data_201812_fault_A_df['target'] = 1
smartlog_data_201812_fault_B_df['target'] = 1
print(len(smartlog_data_201812_fault_A_df))
print(len(smartlog_data_201812_fault_B_df))
smartlog_data_201812_fault_B_df

713
943


,Unnamed: 0,serial_number,manufacturer,model,smart_1_normalized,smart_1raw,smart_3_normalized,smart_3raw,smart_4_normalized,smart_4raw,...,smart_199_normalized,smart_199raw,smart_240_normalized,smart_240raw,smart_241_normalized,smart_241raw,smart_242_normalized,smart_242raw,dt,target
1,250654,disk_59612,A,2,84.0,230420736.0,85.0,0.0,100.0,75.0,...,200.0,0.0,100.0,2.362619e+14,100.0,4.454362e+10,100.0,7.587843e+09,2018-12-21,1
2,311443,disk_59612,A,2,79.0,84811168.0,85.0,0.0,100.0,75.0,...,200.0,0.0,100.0,7.259783e+13,100.0,4.609653e+10,100.0,7.628234e+09,2018-12-24,1
4,372677,disk_59612,A,2,79.0,71480992.0,85.0,0.0,100.0,75.0,...,200.0,0.0,100.0,1.378813e+14,100.0,4.760714e+10,100.0,7.840023e+09,2018-12-27,1
10,104589,disk_59612,A,2,81.0,133053096.0,85.0,0.0,100.0,75.0,...,200.0,0.0,100.0,1.179398e+13,100.0,4.565724e+10,100.0,7.621588e+09,2018-12-23,1
14,58713,disk_59612,A,2,82.0,156155400.0,85.0,0.0,100.0,75.0,...,200.0,0.0,100.0,2.642049e+14,100.0,4.519089e+10,100.0,7.609148e+09,2018-12-22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2832,267801,disk_25650,A,2,81.0,131039064.0,86.0,0.0,100.0,12.0,...,200.0,0.0,100.0,2.969540e+13,100.0,7.121554e+10,100.0,4.720543e+10,2018-12-09,1
2833,490075,disk_25650,A,2,83.0,190991072.0,86.0,0.0,100.0,12.0,...,200.0,0.0,100.0,1.700721e+14,100.0,6.914705e+10,100.0,4.546432e+10,2018-12-07,1
2834,10720,disk_25650,A,2,77.0,48321544.0,86.0,0.0,100.0,12.0,...,200.0,0.0,100.0,8.928378e+13,100.0,7.221756e+10,100.0,4.790706e+10,2018-12-10,1
2835,180519,disk_25650,A,2,68.0,6105192.0,86.0,0.0,100.0,12.0,...,200.0,0.0,100.0,2.461016e+13,100.0,6.698227e+10,100.0,4.392377e+10,2018-12-05,1


In [6]:
start_date = pd.to_datetime('2018-12-26')
end_date = pd.to_datetime('2018-12-31')
# 筛选健康磁盘信息
smartlog_data_A_df = smartlog_data_A_df[~smartlog_data_A_df['serial_number'].isin(fault_tag_A_df['fault_time'])]
smartlog_data_B_df = smartlog_data_B_df[~smartlog_data_B_df['serial_number'].isin(fault_tag_B_df['fault_time'])]
smartlog_data_A_df['dt'] = pd.to_datetime(smartlog_data_A_df['dt'], format='%Y%m%d')
smartlog_data_B_df['dt'] = pd.to_datetime(smartlog_data_B_df['dt'], format='%Y%m%d')
smartlog_data_A_df = smartlog_data_A_df[(smartlog_data_A_df['dt'] >= start_date) & (smartlog_data_A_df['dt'] <= end_date)]
smartlog_data_B_df = smartlog_data_B_df[(smartlog_data_B_df['dt'] >= start_date) & (smartlog_data_B_df['dt'] <= end_date)]
# 将健康磁盘的数据与故障磁盘数据数目保持一直
N = 150  # 用你想要的磁盘数量替换
unique_disks_A = smartlog_data_A_df['serial_number'].unique()[:N]
unique_disks_B = smartlog_data_B_df['serial_number'].unique()[:N]
smartlog_data_A_df = smartlog_data_A_df[smartlog_data_A_df['serial_number'].isin(unique_disks_A)]
smartlog_data_B_df = smartlog_data_B_df[smartlog_data_B_df['serial_number'].isin(unique_disks_B)]
# 添加标签
smartlog_data_A_df['target'] = 0
smartlog_data_B_df['target'] = 0
print(len(smartlog_data_A_df))
print(len(smartlog_data_B_df))
smartlog_data_A_df

891
895


,Unnamed: 0,serial_number,manufacturer,model,smart_1_normalized,smart_1raw,smart_3_normalized,smart_3raw,smart_4_normalized,smart_4raw,...,smart_195_normalized,smart_195raw,smart_197_normalized,smart_197raw,smart_198_normalized,smart_198raw,smart_199_normalized,smart_199raw,dt,target
0,0,disk_59274,A,1,82.0,200026853.0,95.0,0.0,100.0,13.0,...,36.0,200026853.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-26,0
5,5,disk_59284,A,1,78.0,69840115.0,96.0,0.0,100.0,14.0,...,56.0,69840115.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-29,0
9,9,disk_59287,A,1,83.0,217702651.0,96.0,0.0,100.0,13.0,...,23.0,217702651.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-29,0
10,10,disk_59289,A,1,82.0,196901095.0,96.0,0.0,100.0,13.0,...,59.0,196901095.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-28,0
13,13,disk_59294,A,1,82.0,170317941.0,96.0,0.0,100.0,29.0,...,54.0,170317941.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-26,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4100642,585526,disk_59677,A,1,78.0,74231831.0,96.0,0.0,100.0,14.0,...,63.0,74231831.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-30,0
4100674,585558,disk_59744,A,1,81.0,145032042.0,95.0,0.0,100.0,14.0,...,21.0,145032042.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-30,0
4100688,585572,disk_59766,A,1,78.0,60170577.0,96.0,0.0,100.0,15.0,...,65.0,60170577.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-29,0
4100724,585608,disk_5987,A,1,83.0,220736046.0,95.0,0.0,100.0,8.0,...,48.0,220736046.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-30,0


In [7]:
# 合并数据集
smartlog_data_A_all_df = pd.concat([smartlog_data_A_df,smartlog_data_201812_fault_A_df])
smartlog_data_B_all_df = pd.concat([smartlog_data_B_df,smartlog_data_201812_fault_B_df])
smartlog_data_A_all_df

,Unnamed: 0,serial_number,manufacturer,model,smart_1_normalized,smart_1raw,smart_3_normalized,smart_3raw,smart_4_normalized,smart_4raw,...,smart_195_normalized,smart_195raw,smart_197_normalized,smart_197raw,smart_198_normalized,smart_198raw,smart_199_normalized,smart_199raw,dt,target
5,5,disk_59284,A,1,78.0,69840115.0,96.0,0.0,100.0,14.0,...,56.0,69840115.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-29,0
9,9,disk_59287,A,1,83.0,217702651.0,96.0,0.0,100.0,13.0,...,23.0,217702651.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-29,0
10,10,disk_59289,A,1,82.0,196901095.0,96.0,0.0,100.0,13.0,...,59.0,196901095.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-28,0
15,15,disk_59299,A,1,81.0,146003577.0,96.0,0.0,100.0,14.0,...,63.0,146003577.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-29,0
18,18,disk_59300,A,1,78.0,64843235.0,95.0,0.0,100.0,11.0,...,46.0,64843235.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-27,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,430809,disk_145636,A,1,81.0,118630444.0,95.0,0.0,100.0,16.0,...,63.0,118630444.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-05,1
2385,529268,disk_145636,A,1,80.0,104656645.0,95.0,0.0,100.0,16.0,...,62.0,104656645.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-03,1
2386,529269,disk_145636,A,1,80.0,109052224.0,95.0,0.0,100.0,16.0,...,56.0,109052224.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-04,1
2387,467459,disk_145636,A,1,81.0,129379353.0,95.0,0.0,100.0,16.0,...,63.0,129379353.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-12-07,1


##### 皮尔逊相关系数 

In [8]:
# 类型A的相关性
correlations_A = smartlog_data_A_all_df.corr()['target']
sorted_correlations_A = correlations_A.sort_values(ascending=False)
print(sorted_correlations_A)

target                  1.000000
Unnamed: 0              0.192531
smart_5raw              0.137441
smart_4raw              0.121917
smart_192raw            0.115747
smart_12raw             0.113747
smart_198raw            0.094459
smart_197raw            0.094459
smart_9_normalized      0.071878
smart_3_normalized      0.069711
smart_190raw            0.060080
smart_194raw            0.060080
smart_194_normalized    0.060080
smart_7raw              0.054530
smart_188raw            0.053614
smart_187raw            0.049081
smart_193_normalized    0.036415
smart_189_normalized    0.015029
smart_195raw            0.013240
smart_1raw              0.013240
smart_188_normalized   -0.031516
smart_193raw           -0.047650
smart_189raw           -0.049767
smart_187_normalized   -0.052349
smart_190_normalized   -0.060080
smart_9raw             -0.063352
smart_1_normalized     -0.065841
smart_197_normalized   -0.091692
smart_198_normalized   -0.091692
smart_195_normalized   -0.093205
smart_5_no

In [9]:
# 类型B的相关性
correlations_B = smartlog_data_B_all_df.corr()['target']
sorted_correlations_B = correlations_B.sort_values(ascending=False)
print(sorted_correlations_B)

target                  1.000000
smart_9_normalized      0.440649
smart_191_normalized    0.304597
smart_192raw            0.201166
smart_5raw              0.194170
smart_194raw            0.172813
smart_194_normalized    0.172813
smart_190raw            0.172813
smart_199raw            0.134886
smart_195_normalized    0.125238
smart_188raw            0.120487
smart_4raw              0.086506
smart_12raw             0.080405
smart_193_normalized    0.070106
smart_187raw            0.062595
smart_3_normalized      0.057814
smart_195raw            0.039119
smart_1raw              0.039119
smart_198raw            0.036894
smart_197raw            0.036894
smart_184raw            0.021657
Unnamed: 0              0.019895
smart_240raw            0.009316
smart_184_normalized   -0.021657
smart_197_normalized   -0.021657
smart_198_normalized   -0.021657
smart_5_normalized     -0.021657
smart_188_normalized   -0.037533
smart_1_normalized     -0.060734
smart_187_normalized   -0.062595
smart_193r